In [ ]:
import os
import gnssrefl.gps as g
import gnssrefl.rinex2snr as rnx
import gnssrefl.quickLook_function as quick
import gnssrefl.gnssir as guts
import json
import pandas as pd 
import check_parameters
import requests
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(style="whitegrid");
import re
from datetime import datetime
import numpy as np

# making sure that env variables are set - if they are then nothing will print to screen
g.check_environ_variables()

%matplotlib inline

### Island Park, Idaho
  
**Station Name:**  p360

**Location:**  Island Park, Idaho

**Archive:**  [UNAVCO](http://www.unavco.org)

**DOI:**  [https://doi.org/10.7283/T5DB7ZR3](https://doi.org/10.7283/T5DB7ZR3)

**Ellipsoidal Coordinates:**

- Latitude:  44.31785

- Longitude: -111.45068

- Height: 1857.861 m

[Station Page at UNAVCO](https://www.unavco.org/instrumentation/networks/status/nota/overview/P360)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/P360.sta)

[Station Page at PBO H2O](http://cires1.colorado.edu/portal/index.php?product=snow&station=p360)

[Google Map Link](https://goo.gl/maps/EcTkbHjaSaWp4d8H9)

<img src="https://gnss-reflections.org/static/images/P360.jpg" width="500">

## Data Summary

Station p360 is located to the west of Yellowstone National Park, near the town of Island Park in Idaho.  At an elevation of ~1858 m (6096 ft), winter snowfall can be frequent and heavy.

The site is operated by UNAVCO and has been recording multi-GNSS data since March 2020. Before that time, only
the L2C GPS data are of reliable quality. P360 was part of [PBO H2O](http://cires1.colorado.edu/portal/).

More information on the station can be obtained from the [GNSS-IR Web App](https://gnss-reflections.org/fancy6?example=p360), where p360 is one of the test cases for snow depth. The station is in a flat, grassy plain with no obstacles or changes in topography, so elevation and azimuth masks are not required.  The web app has a tab for [Reflection Zone Mapping](https://gnss-reflections.org/rzones?station=p360&lat=0.0&lon=0.0&height=0.0&msl=off&RH=2&eang=1&azim1=0&azim2=360) where reflection zones for selected elevation angles are plotted.  The Reflection Zone Mapping allows for experimentation with reflector heights, elevation angles, and azimuth masks.

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/api?example=p360" width="1000" height="900"></iframe>

## Take a Quick Look at the Data

**Make SNR File**

First you need to make an SNR file. Ordinarily one can only access L2C data at
this site using the 1-sec data. To support gnssrefl users, UNAVCO
has created special RINEX files with L2C SNR data in them at a friendlier sample
rate, 15 sec. Use the special archive:

In [ ]:
station = 'p360'
year = 2017
doy = 290

lat = 44.3179
long = -111.4507
height = 1857.865

In [ ]:
args = check_parameters.rinex2snr(station,year,doy, translator='hybrid', archive='special')
rnx.run_rinex2snr(**args)

**quickLook** analyzes the [spectral characteristics](https://www.unavco.org/gitlab/gnss_reflectometry/gnssrefl/-/blob/test/docs/quickLook_desc.md) of the SNR data.

In [ ]:
# making a plotting function for the quicklook function
def quicklook_results(args, values):
    freq = {1:'L1', 20: 'L2C'}
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10,10))
    quadrants = ['NW', 'NE', 'SW', 'SE']
    axes = [ax[0,0], ax[0,1], ax[1,0], ax[1,1]]

    for i, quadrant in enumerate(quadrants):
        satellites = values[quadrant].keys()
        fail_satellites = values[f'f{quadrant}'].keys()

        for failsat in fail_satellites:
            axes[i].plot(values[f'f{quadrant}'][failsat][0], values[f'f{quadrant}'][failsat][1], color='lightgrey') 
        for sat in satellites:
            axes[i].plot(values[quadrant][sat][0], values[quadrant][sat][1])

    ax[0,0].set_title('Northwest', size=14)
    ax[0,1].set_title('Northeast',size=14)
    ax[1,0].set_title('Southwest', size=14)
    ax[1,1].set_title('Southeast', size=14)

    for ax in axes:
        ax.set_xlabel('reflector height (m)', size=14)
        ax.set_ylabel('volts/volts', size=14)
        ax.grid()
    
    fig.suptitle(f'GNSS Station {args["station"].upper()}, {args["year"]} doy {args["doy"]}, freq {freq[args["f"]]}, elevation angles {args["e1"]}-{args["e2"]} \n', size=16)
    fig.tight_layout()
    plt.show()
    
    
def quicklook_metrics(args, values):
#     fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(10,10), sharex=True)
    quadrants = ['NW', 'NE', 'SW', 'SE']
    
    # re-organizing the data in a plotting friendly format
    success_data = {'Azimuth': [], 'Reflector Height': [], 'Peak to Noise':[], 'Amplitude': []}
    fail_data =  {'Azimuth': [], 'Reflector Height': [], 'Peak to Noise': [], 'Amplitude': []}
    
    for i, quadrant in enumerate(quadrants):
        for j in values[quadrant].keys():
            success_data['Azimuth'].append(datakeys[quadrant][j][0])
            success_data['Reflector Height'].append(datakeys[quadrant][j][1])
            success_data['Peak to Noise'].append(datakeys[quadrant][j][5])
            success_data['Amplitude'].append(datakeys[quadrant][j][4])
        for k in values[f'f{quadrant}'].keys():
            fail_data['Azimuth'].append(datakeys[f'f{quadrant}'][k][0])
            fail_data['Reflector Height'].append(datakeys[f'f{quadrant}'][k][1])
            fail_data['Peak to Noise'].append(datakeys[f'f{quadrant}'][k][5])
            fail_data['Amplitude'].append(datakeys[f'f{quadrant}'][k][4])

    return pd.DataFrame(success_data), pd.DataFrame(fail_data)   

In [ ]:
# To understand what quicklook returns, you can uncomment the next line of code to learn more about this function 
# and it's default parameters
# check_parameters.quicklook?

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

The default return is for L1. This plot confirms our concerns about the quality of the L1 data.
It also suggests the southern quadrants are preferred to other quadrants.

and for L2C:

In [ ]:
args = check_parameters.quicklook(station, year, doy=doy, f=20)
values, datakeys = quick.quickLook_function(**args)
quicklook_results(args, values)

These reflector height retrievals are far superior to the L1 data. The southern quadrants give more
consistent retrievals than for the north. This is confirmed in the QC plot show here:

In [ ]:
success, fail = quicklook_metrics(args, datakeys)
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(10,10), sharex=True)
fig.suptitle(f'QuickLook Retrieval Metrics: {args["station"]} GPS L1', size=16)

for i, ax in enumerate(axes):
    g = sns.scatterplot(x='Azimuth',y=success.columns[i+1], data=success, ax=ax, label='good')
    g = sns.scatterplot(x='Azimuth',y=fail.columns[i+1], data=fail, ax=ax, color='lightgrey', label='bad')
    
axes[0].legend(loc='upper right')
avg_rh = np.mean(success['Reflector Height'])
# qc_val_peak2noise = round(min(success['Peak to Noise']))
# axes[1].axhline(qc_val_peak2noise, linestyle='--', color='black', label='QC value used')
# qc_val_amp = round(min(success['Amplitude']))
# axes[2].axhline(qc_val_amp, linestyle='--', color='black', label='QC value used')
print(f'Average reflector height value: {avg_rh:.1f}')
# print('QC value for peak to noise:', qc_val_peak2noise)
# print('QC value for amplitude:', qc_val_amp)

plt.tight_layout()
plt.show()

## Analyze the Data

First we will set the analysis paramaters using **make_json_input**. This analysis will use the L2C frequency and
will use QC metrics derived from the previous plot (for peak to noise ratio and amplitude).

In [ ]:
# To understand what make_json returns, you can uncomment the next line of code to learn more about this function 
# and it's default parameters
check_parameters.make_json?

In [ ]:
check_parameters.make_json(station, lat, long, height, peak2noise=3.2, ampl=8, l2c=True)

We will hand edit the json file to disallow 0-90 degrees in azimuth.

In [ ]:
# This is the json file that was created
json_file = 'input/p360.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    file['azval'] = [90, 180,180,270,270,360]

os.remove(json_file)
with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)
    
with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

We then make SNR files to encompass one water year:

In [ ]:
args = check_parameters.rinex2snr(station, year=2017, doy=245, doy_end=365, archive='special', translator='hybrid')
rnx.run_rinex2snr(**args)

In [ ]:
args = check_parameters.rinex2snr(station, year=2018, doy=1, doy_end=145, archive='special', translator='hybrid')
rnx.run_rinex2snr(**args)

Then we run gnssir to calculate the reflector heights for 2017/2018. Because the code
only creates results if the SNR file exists, we can use the year_end and doy_end settings.

In [ ]:
year = 2017
doy = 1
doy_end = 366
year_end = 2018
plt=False
args = check_parameters.gnssir(station, year, doy, doy_end=doy_end, year_end=year_end, plt=plt, screenstats=False)
year_list = list(range(year, args['year_end'] + 1))
doy_list = list(range(doy, args['doy_end'] + 1))
for year in year_list:
    args['args']['year'] = year
    for doy in doy_list:
        args['args']['doy'] = doy
        guts.gnssir_guts(**args['args'])

## Derive Snow Accumulation from Reflector Heights

Use the daily_avg utility with a relatively low number of required satellite tracks (12) 
and 0.25 meter median filter to remove large outliers:


In [ ]:
check_parameters.daily_avg(station, medfilter=.25, ReqTracks=12, plt2screen=False, txtfile='p360-dailyavg.txt')


In [ ]:
import matplotlib.pyplot as plt

def read_allRH_file(filepath, regex):
    data = {'dates': [], 'rh': []}
    #read daily average reflector heights
    with open(f'{refl_code_dir}{filepath}', 'r') as myfile:
        file = myfile.read()
        matches = re.finditer(regex, file, flags=re.MULTILINE)

        for match in matches:
            ydoy = f'{int(match.group("year"))}-{int(match.group("doy"))}'
            date = datetime.strptime(ydoy, '%Y-%j').date()
            data['dates'].append(date)
            data['rh'].append(float(match.group('rh')))
            
    return data

regex = '^ (?P<year>[ \d]+) +(?P<doy>[\d]+) +(?P<rh>[\d|-|.]+)'
filepath = f'/Files/{station}_allRH.txt'
data = read_allRH_file(filepath, regex)

df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
plt.figure(figsize=(10,8))
g = sns.scatterplot(x='dates', y='rh', data=df, hue='dates', palette='colorblind', legend=False)
g.set_ylim(2.4, .8)
g.set_ylabel('Reflector Height (m)');

This plot makes it clear that there are data outages at p360, particular in the winter months.
This is due to the way the site was built. Power was provided by solar panels and batteries. When
snow covered the solar panels (see the photograph above), the site would soon lose power when
the batteries ran down. When the snow on the panels melted, the site would track again.
This next plot shows the number of reflector heights each day. While they are consistent
in the fall and later spring, there is significant variation in December, January, and February.

In [ ]:
plt.figure(figsize=(10,8))
df_group = df.groupby(['dates']).agg(['count'])
g = sns.scatterplot(data=df_group)
g.set_title('Number of values used in the daily average', size=16);

The variation in retrievals in the winter is due to poor data retrieval in those winter months.
This next plot shows the number of total observations in the RINEX file each day. You can see the
correlation between the number of tracks available for the daily average and the number of
observations in the file. For sites with more reliable power, there will not be
data outages during the winter.

Finally, the daily average RH for the water year:

In [ ]:
regex = '^ (?P<year>[ \d]+) +(?P<doy>[\d]+) +(?P<rh>[\d|-|.]+)'
filepath = f'/Files/{station}-dailyavg.txt'
data = read_allRH_file(filepath, regex)
df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])

plt.figure(figsize=(10,8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False)
g.set_ylim(2.2,1)
g.set_ylabel('Reflector Height (m)')
g.set_title(f'{station}: Daily Mean Reflector Height', size=16);